# Setup environment
This notebook loads environment variables and saves them for later use in any other notebook.

## Star GitHub repo

In [ ]:
%%html

<a class="github-button" href="https://github.com/aws-samples/amazon-sagemaker-secure-mlops" data-color-scheme="no-preference: light; light: light; dark: dark;" data-icon="octicon-star" data-size="large" data-show-count="true" aria-label="Star Amazon SageMaker secure MLOps on GitHub">Star</a>
<script async defer src="https://buttons.github.io/buttons.js"></script>

## _Click This Button ^^ Above ^^_

## Get domain id

In [ ]:
import os
import json

NOTEBOOK_METADATA_FILE = "/opt/ml/metadata/resource-metadata.json"
domain_id = None

if os.path.exists(NOTEBOOK_METADATA_FILE):
    with open(NOTEBOOK_METADATA_FILE, "rb") as f:
        domain_id = json.loads(f.read()).get('DomainId')
        print(f"SageMaker domain id: {domain_id}")

%store domain_id

## Define `get_environment` function

In [ ]:
import boto3

sm = boto3.client("sagemaker")
ssm = boto3.client("ssm")

def get_environment(ssm_params):
    r = sm.describe_domain(
            DomainId=[
                d["DomainId"] for d in sm.list_domains()["Domains"] if boto3.Session().region_name in d["DomainArn"]
            ][0]
        )
    del r["ResponseMetadata"]
    del r["CreationTime"]
    del r["LastModifiedTime"]
    r = {**r, **r["DefaultUserSettings"]}
    del r["DefaultUserSettings"]

    i = {
        **r,
        **{t["Key"]:t["Value"] 
            for t in sm.list_tags(ResourceArn=r["DomainArn"])["Tags"] 
            if t["Key"] in ["EnvironmentName", "EnvironmentType"]}
    }

    for p in ssm_params:
        try:
            i[p["VariableName"]] = ssm.get_parameter(Name=f"{i['EnvironmentName']}-{i['EnvironmentType']}-{p['ParameterName']}")["Parameter"]["Value"]
        except:
            i[p["VariableName"]] = ""

    return i

def get_session(region, default_bucket):
    """Gets the sagemaker session based on the region.

    Args:
        region: the aws region to start the session
        default_bucket: the bucket to use for storing the artifacts

    Returns:
        sagemaker.session.Session instance
    """

    boto_session = boto3.Session(region_name=region)

    sagemaker_client = boto_session.client("sagemaker")
    runtime_client = boto_session.client("sagemaker-runtime")
    return sagemaker.session.Session(
        boto_session=boto_session,
        sagemaker_client=sagemaker_client,
        sagemaker_runtime_client=runtime_client,
        default_bucket=default_bucket,
    )

## Load and output environment variables

In [ ]:
# Dynamically load environmental SSM parameters - provide the list of the variables to load from SSM parameter store
ssm_parameters = [
    {"VariableName":"DataBucketName", "ParameterName":"data-bucket-name"},
    {"VariableName":"ModelBucketName", "ParameterName":"model-bucket-name"},
    {"VariableName":"S3VPCEId", "ParameterName":"s3-vpce-id"},
    {"VariableName":"S3KmsKeyId", "ParameterName":"kms-s3-key-arn"},
    {"VariableName":"EbsKmsKeyArn", "ParameterName":"kms-ebs-key-arn"},
    {"VariableName":"PipelineExecutionRole", "ParameterName":"sm-pipeline-execution-role-arn"},
    {"VariableName":"ModelExecutionRole", "ParameterName":"sm-model-execution-role-name"},
    {"VariableName":"StackSetExecutionRole", "ParameterName":"stackset-execution-role-name"},
    {"VariableName":"StackSetAdministrationRole", "ParameterName":"stackset-administration-role-arn"},
    {"VariableName":"StagingAccountList", "ParameterName":"staging-account-list"},
    {"VariableName":"ProdAccountList", "ParameterName":"production-account-list"},
    {"VariableName":"EnvTypeStagingName", "ParameterName":"env-type-staging-name"},
    {"VariableName":"EnvTypeProdName", "ParameterName":"env-type-prod-name"},
]

env_data = get_environment(ssm_params=ssm_parameters)
print(f"Environment data:\n{json.dumps(env_data, indent=2)}")

## Setup network configuration

In [ ]:
from sagemaker.network import NetworkConfig

# Network Isolation mode; default is off
enable_network_isolation = False

# Intercontainer traffic encryption is on
encrypt_inter_container_traffic = True

network_config = NetworkConfig(
        enable_network_isolation=enable_network_isolation, 
        security_group_ids=env_data["SecurityGroups"],
        subnets=env_data["SubnetIds"],
        encrypt_inter_container_traffic=encrypt_inter_container_traffic)

## Store environment variables

In [ ]:
security_group_ids = env_data["SecurityGroups"]
subnets = env_data["SubnetIds"]
processing_role = env_data["ExecutionRole"]
execution_role = env_data["ExecutionRole"]
pipeline_role = env_data["PipelineExecutionRole"]
model_execution_role = env_data["ModelExecutionRole"]
data_bucket = env_data["DataBucketName"]
model_bucket = env_data["ModelBucketName"]
ebs_key_id = env_data["EbsKmsKeyArn"]
s3_key_id = env_data["S3KmsKeyId"]
env_name = env_data["EnvironmentName"]
env_type = env_data["EnvironmentType"]
env_type_staging_name = env_data["EnvTypeStagingName"]
env_type_prod_name = env_data["EnvTypeProdName"]

%store security_group_ids
%store subnets
%store processing_role
%store execution_role
%store model_execution_role
%store pipeline_role
%store data_bucket
%store model_bucket
%store ebs_key_id
%store s3_key_id
%store network_config
%store env_name
%store env_type
%store env_type_staging_name
%store env_type_prod_name

In [ ]:
setup_done = True
%store setup_done

%store

## Release resources

In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>